In [4]:
from sampler_gclusters import sampler_gclusters
import numpy as np
import jax
import jax.numpy as jnp
from jax import random
from tqdm.auto import tqdm
from pathlib import Path
Path("results").mkdir(exist_ok=True)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from all_tests import hsicfuse_test, hsic_test
from all_tests import hsicagginc1_test, hsicagginc100_test, hsicagginc200_test, hsicaggincquad_test

### Vary difficulty

In [11]:
repetitions          = 200
Ls                   = (1, 2, 4, 8, 16, 32)
sample_size          = 500  

tests = (hsicfuse_test, hsic_test, hsicagginc1_test, hsicagginc100_test, hsicagginc200_test, hsicaggincquad_test, )
outputs = jnp.zeros((len(tests), len(Ls), repetitions))
outputs = outputs.tolist()
key = random.PRNGKey(42)
seed = 42
for s in (range(len(Ls))):
    L = Ls[s]
    for i in (range(repetitions)):
        key, subkey = random.split(key)
        X, Y = sampler_gclusters(key=key, L=L, theta=0, N=sample_size, permute=False)
        key, subkey = random.split(key)
        seed += 1
        for t in range(len(tests)):
            test = tests[t]
            outputs[t][s][i] = test(
                    X,
                    Y,
                    subkey,
                    seed,
                )

output = jnp.mean(jnp.array(outputs), -1) # the last dimension is eliminated

jnp.save("results/gclusters_vary_dif.npy", output)
jnp.save("results/gclusters_vary_dif_x_axis.npy", Ls)

print("Ls :", Ls)
print("sample size :", sample_size)
for t in range(len(tests)):
    print(" ")
    print(tests[t])
    print(output[t])

Ls : (1,)
sample size : 500
 
<function hsic_test at 0x7fdff316bf70>
[0.]
